In [1]:
Lsamples=30000;
prob_void=0.15;
target_label=['negative','neutro','pain','positive'];
output_base_dir='../../BER2024/BER2024-FUSION'
dataset_name='dummy'
csv_train_filename='train.csv'
csv_test_filename='test.csv'

In [2]:
import random 

def flip_coin(p):
    return True if random.random() < p else False

In [3]:
import numpy as np

def create_vector4(N):
    if N < 0 or N > 3:
        raise ValueError("N deve estar entre 0 e 3, inclusive.")
    
    # Gera 3 valores aleatórios
    random_values = np.random.rand(3)
    
    # Gera um valor aleatório que é maior que todos os 3 valores gerados
    max_value = random_values.max() + np.random.rand()
    
    # Cria o vetor de 4 elementos com o valor máximo na posição N
    vector = np.insert(random_values, N, max_value)
    
    return vector/np.sum(vector);

# Exemplo de uso
N = 0
vector = create_vector4(N)
print(vector)


[0.32764816 0.23658177 0.24997471 0.18579536]


In [4]:
import os

output_dir=os.path.join(output_base_dir,dataset_name,'L'+str(Lsamples)+'_p'+str(prob_void))

os.makedirs(output_dir,exist_ok = True);

In [5]:

import csv
import json
from tqdm import tqdm

def create_dummy_dataset(csv_filename,Lsamples,target_label,drop1_prob=0.0):
    Info=dict();
    for label in target_label:
        Info[label]=0;
    
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Escreve o cabeçalho (opcional)
        header = ['d'+str(n) for n in range(3*len(target_label))]+[ 'label'];
        writer.writerow(header)
        
        for l in tqdm(range(Lsamples)):
            
            ID=random.randint(0, len(target_label)-1);
        
            vector1 = create_vector4(ID);
            vector2 = create_vector4(ID);
            vector3 = create_vector4(ID);

            if drop1_prob>0.0:
                if flip_coin(drop1_prob):
                    id_drop=random.randint(0,2);
                    if id_drop==0:
                        vector1=np.zeros(4);
                    if id_drop==1:
                        vector2=np.zeros(4);
                    if id_drop==2:
                        vector3=np.zeros(4);
            
            concatenated = np.concatenate((vector1, vector2, vector3))
            label=target_label[ID];

            Info[label]=Info[label]+1;
    
            # Converte o vetor em lista e adiciona a etiqueta no final
            row = list(concatenated) + [label];
            
            # Escreve a linha no arquivo CSV
            writer.writerow(row)
    # Salvar o dicionário como um arquivo JSON
    with open(csv_filename+'.json', 'w') as json_file:
        json.dump(Info, json_file, indent=4, sort_keys=True)

In [6]:
csv_filename=os.path.join(output_dir,csv_train_filename);
create_dummy_dataset(csv_filename,Lsamples,target_label,drop1_prob=prob_void);

100%|███████████████████████████████████| 30000/30000 [00:03<00:00, 9884.36it/s]


In [7]:
csv_filename=os.path.join(output_dir,csv_test_filename);
create_dummy_dataset(csv_filename,Lsamples,target_label,drop1_prob=prob_void);

100%|███████████████████████████████████| 30000/30000 [00:03<00:00, 9770.83it/s]
